In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

from src.paths import TRANSFORMED_DATA_DIR 

df = pd.read_parquet(TRANSFORMED_DATA_DIR/ 'tabular_data2.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89300,3.0,0.0,2.0,3.0,2.0,3.0,13.0,8.0,9.0,9.0,...,6.0,5.0,3.0,1.0,6.0,1.0,3.0,2022-12-27,265,3.0
89301,6.0,4.0,0.0,0.0,2.0,0.0,14.0,7.0,8.0,4.0,...,4.0,2.0,1.0,2.0,2.0,2.0,8.0,2022-12-28,265,1.0
89302,7.0,2.0,3.0,4.0,7.0,4.0,10.0,9.0,7.0,11.0,...,2.0,3.0,5.0,1.0,1.0,0.0,8.0,2022-12-29,265,3.0
89303,6.0,5.0,4.0,3.0,0.0,3.0,11.0,12.0,9.0,10.0,...,3.0,3.0,1.0,2.0,0.0,1.0,2.0,2022-12-30,265,7.0


In [3]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(56710, 674)
y_test.shape=(56710,)


In [4]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [5]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2023-07-18 20:30:24,120] A new study created in memory with name: no-name-02e5bc20-f8d9-4f24-b4c8-dc8124838e9a


[LightGBM] [Warning] feature_fraction is set=0.38982033750179834, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.38982033750179834
[LightGBM] [Warning] bagging_fraction is set=0.5479324829421908, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5479324829421908
[LightGBM] [Warning] feature_fraction is set=0.38982033750179834, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.38982033750179834
[LightGBM] [Warning] bagging_fraction is set=0.5479324829421908, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5479324829421908


[I 2023-07-18 20:30:37,622] Trial 0 finished with value: 3.395607399850668 and parameters: {'num_leaves': 78, 'feature_fraction': 0.38982033750179834, 'bagging_fraction': 0.5479324829421908, 'min_child_samples': 43}. Best is trial 0 with value: 3.395607399850668.


[LightGBM] [Warning] feature_fraction is set=0.5880488498687282, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5880488498687282
[LightGBM] [Warning] bagging_fraction is set=0.7973918045217372, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7973918045217372
[LightGBM] [Warning] feature_fraction is set=0.5880488498687282, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5880488498687282
[LightGBM] [Warning] bagging_fraction is set=0.7973918045217372, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7973918045217372


[I 2023-07-18 20:31:02,931] Trial 1 finished with value: 3.341458916027606 and parameters: {'num_leaves': 209, 'feature_fraction': 0.5880488498687282, 'bagging_fraction': 0.7973918045217372, 'min_child_samples': 21}. Best is trial 1 with value: 3.341458916027606.


...

[I 2023-07-18 20:31:25,611] Trial 2 finished with value: 3.8497433533274656 and parameters: {'num_leaves': 130, 'feature_fraction': 0.7516950518970635, 'bagging_fraction': 0.251574805658576, 'min_child_samples': 98}. Best is trial 1 with value: 3.341458916027606.


[LightGBM] [Warning] feature_fraction is set=0.286074405149185, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.286074405149185
[LightGBM] [Warning] bagging_fraction is set=0.7377817020960127, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7377817020960127
[LightGBM] [Warning] feature_fraction is set=0.286074405149185, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.286074405149185
[LightGBM] [Warning] bagging_fraction is set=0.7377817020960127, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7377817020960127


[I 2023-07-18 20:31:34,229] Trial 3 finished with value: 3.433099068341141 and parameters: {'num_leaves': 38, 'feature_fraction': 0.286074405149185, 'bagging_fraction': 0.7377817020960127, 'min_child_samples': 68}. Best is trial 1 with value: 3.341458916027606.


[LightGBM] [Warning] feature_fraction is set=0.9261474863765564, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9261474863765564
[LightGBM] [Warning] bagging_fraction is set=0.5358181428585771, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5358181428585771
[LightGBM] [Warning] feature_fraction is set=0.9261474863765564, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9261474863765564
[LightGBM] [Warning] bagging_fraction is set=0.5358181428585771, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5358181428585771


[I 2023-07-18 20:31:43,632] Trial 4 finished with value: 3.3397028447528823 and parameters: {'num_leaves': 19, 'feature_fraction': 0.9261474863765564, 'bagging_fraction': 0.5358181428585771, 'min_child_samples': 22}. Best is trial 4 with value: 3.3397028447528823.


In [6]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 19, 'feature_fraction': 0.9261474863765564, 'bagging_fraction': 0.5358181428585771, 'min_child_samples': 22}


In [7]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)
        

[LightGBM] [Warning] feature_fraction is set=0.9261474863765564, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9261474863765564
[LightGBM] [Warning] bagging_fraction is set=0.5358181428585771, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5358181428585771


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001E6EB4B50D0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.5358181428585771,
                               feature_fraction=0.9261474863765564,
                               min_child_samples=22, num_leaves=19))])

In [8]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae = :.4f}')

test_mae = 2.5830


In [9]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 3979,
    predictions=pd.Series(predictions)
)

In [10]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 200,
    predictions=pd.Series(predictions)
)

In [11]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 1000,
    predictions=pd.Series(predictions)
)

In [12]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 900,
    predictions=pd.Series(predictions)
)

In [13]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 450,
    predictions=pd.Series(predictions)
)

In [14]:
from src.plot import plot_one_sample
plot_one_sample(
    features = X_test,
    targets = y_test,
    example_id= 6000,
    predictions=pd.Series(predictions)
)